Builds GIFs from your exported PNGs. Handles mixed sizes by padding onto a common canvas.


In [4]:
# 09_GIF_PolicyMaps.ipynb
# Builds GIFs from your exported PNGs, padding frames to a common canvas,
# and saves ALL GIFs into one folder.

import os, glob, re
import numpy as np
from PIL import Image
import imageio.v2 as imageio

# ---------- Output folder for ALL GIFs
OUTPUT_GIF_DIR = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\gifs"
os.makedirs(OUTPUT_GIF_DIR, exist_ok=True)

# ---------- Your source folders & patterns
TARGETS = [
    {
        "name": "Kriged_Categorical",
        "in_dir": r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local\Kriged Categorical",
        "patterns": ["date_2025-*_kriged_categorical.png", "*_kriged_categorical.png"],
        "out_gif": "pm25_kriged_categorical.gif",
        "duration_sec": 0.8,
    },
    {
        "name": "Kriged_Continuous",
        "in_dir": r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local\Kriged Continuous",
        "patterns": ["date_2025-*_kriged_continuous.png", "*_kriged_continuous.png"],
        "out_gif": "pm25_kriged_continuous.gif",
        "duration_sec": 0.8,
    },
    {
        "name": "Kriged_Contours",
        "in_dir": r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours",
        "patterns": ["date_2025-*_kriged_localContours.png", "*_kriged_localContours.png"],
        "out_gif": "pm25_kriged_localContours.gif",
        "duration_sec": 0.8,
    },
    {
        "name": "Uncertainty",
        "in_dir": r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\uncertainty",
        "patterns": ["date_2025-*_uncertainty.png", "*_uncertainty.png"],
        "out_gif": "pm25_uncertainty.gif",
        "duration_sec": 0.8,
    },
    {
        "name": "AQI_Points_Local",
        "in_dir": r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\aqi_points_local",
        "patterns": ["date_2025-*_aqi_points_local.png", "*_aqi_points_local.png"],
        "out_gif": "pm25_aqi_points_local.gif",
        "duration_sec": 0.8,
    },
]

# ---------- Utilities
def extract_date_key(path):
    """Sortable key from filename; prefers 'date_YYYY-MM-DD'."""
    name = os.path.basename(path)
    for rx in (r"date_(\d{4}-\d{2}-\d{2})", r"(\d{4}-\d{2}-\d{2})"):
        m = re.search(rx, name)
        if m: return m.group(1)
    return name

def find_pngs(in_dir, patterns):
    files = []
    if os.path.isdir(in_dir):
        for pat in patterns:
            files.extend(glob.glob(os.path.join(in_dir, pat)))
    return sorted(set(files), key=extract_date_key)

def pad_to_canvas(img: Image.Image, canvas_size, bg=(255, 255, 255)):
    """Center img on a solid-color canvas (no resize/distortion)."""
    canvas = Image.new("RGB", canvas_size, bg)
    w, h = img.size
    x = (canvas_size[0] - w) // 2
    y = (canvas_size[1] - h) // 2
    canvas.paste(img, (x, y))
    return canvas

def make_gif(in_dir, patterns, out_gif_path, duration_sec=0.8):
    pngs = find_pngs(in_dir, patterns)
    print(f"\n[{os.path.basename(in_dir)}] Found {len(pngs)} PNG(s) in:", in_dir)
    if not pngs:
        print("  (None found) — check patterns or run upstream notebooks.")
        return None
    print("  First 5:", [os.path.basename(p) for p in pngs[:5]])

    # Common canvas size
    sizes = []
    for p in pngs:
        with Image.open(p) as im:
            sizes.append(im.size)  # (w, h)
    max_w = max(w for w, h in sizes); max_h = max(h for w, h in sizes)
    target_size = (max_w, max_h)

    # Build frames (pad to common canvas)
    frames = []
    for p in pngs:
        with Image.open(p).convert("RGB") as im:
            framed = pad_to_canvas(im, target_size)
            frames.append(np.array(framed))

    imageio.mimsave(out_gif_path, frames, duration=duration_sec)
    print(f"  ✔ GIF saved: {out_gif_path}  |  frames: {len(frames)}  |  size: {target_size[0]}×{target_size[1]}")
    return out_gif_path

# ---------- Build GIFs to the single OUTPUT_GIF_DIR
out_paths = []
for t in TARGETS:
    save_path = os.path.join(OUTPUT_GIF_DIR, t["out_gif"])
    out = make_gif(t["in_dir"], t["patterns"], save_path, t["duration_sec"])
    if out:
        out_paths.append(out)

print("\nDone. Generated:", len(out_paths), "GIF(s). Saved to:", OUTPUT_GIF_DIR)
for p in out_paths:
    print(" -", p)


[Kriged Categorical] Found 176 PNG(s) in: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local\Kriged Categorical
  First 5: ['date_2025-02-06_kriged_categorical.png', 'date_2025-02-07_kriged_categorical.png', 'date_2025-02-08_kriged_categorical.png', 'date_2025-02-09_kriged_categorical.png', 'date_2025-02-10_kriged_categorical.png']
  ✔ GIF saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\gifs\pm25_kriged_categorical.gif  |  frames: 176  |  size: 1121×937

[Kriged Continuous] Found 176 PNG(s) in: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local\Kriged Continuous
  First 5: ['date_2025-02-06_kriged_continuous.png', 'date_2025-02-07_kriged_continuous.png', 'date_2025-02-08_kriged_continuous.png', 'date_2025-02-09_kriged_continuous.png', 'date_2025-02-10_kriged_continuous.png']
  ✔ GIF saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\gifs\pm25_kriged_continuous.gif  |  frames: 176  |  size: 955×1034

[kriging_local_contours] Found 176 PNG(s) in: C:\Users\kr